In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

In [28]:
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans,AgglomerativeClustering
import warnings 
import sys
if not sys.warnoptions:
    warnings.simplefilter('ignore')
np.random.seed(42)

In [29]:
# ! pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer
from matplotlib.colors import ListedColormap
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D

In [30]:
df_orig = pd.read_csv('marketing_campaign.csv')
df_orig.head(20)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0
5,7446,1967,Master,Together,62513.0,0,1,2013-09-09,16,520,...,6,0,0,0,0,0,0,3,11,0
6,965,1971,Graduation,Divorced,55635.0,0,1,2012-11-13,34,235,...,6,0,0,0,0,0,0,3,11,0
7,6177,1985,PhD,Married,33454.0,1,0,2013-05-08,32,76,...,8,0,0,0,0,0,0,3,11,0
8,4855,1974,PhD,Together,30351.0,1,0,2013-06-06,19,14,...,9,0,0,0,0,0,0,3,11,1
9,5899,1950,PhD,Together,5648.0,1,1,2014-03-13,68,28,...,20,1,0,0,0,0,0,3,11,0


### DATA CLEANING & FEATURE ENGINEERING

In [31]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [32]:
## Income column has 4 null values
## Dt_Customer is of type object and not parsed in datetime


In [33]:
df_orig.dropna(inplace=True)

In [34]:
len(df_orig)

2216

#### Resolving the Dt_Customer column

In [35]:
df_orig['Dt_Customer']
## changing the date to usable format by counting number of days passed

0       2012-09-04
1       2014-03-08
2       2013-08-21
3       2014-02-10
4       2014-01-19
           ...    
2235    2013-06-13
2236    2014-06-10
2237    2014-01-25
2238    2014-01-24
2239    2012-10-15
Name: Dt_Customer, Length: 2216, dtype: object

In [36]:
dates = []
df_orig['Dt_Customer'] =  pd.to_datetime(df_orig['Dt_Customer'])

for cols in df_orig.Dt_Customer:
    upd_date = cols.date()
    dates.append(upd_date)
# taking the max and min from the entire data column will give most recent and older records 
max_date = max(dates)
min_date = min(dates)
max_date,min_date

(datetime.date(2014, 6, 29), datetime.date(2012, 7, 30))

#### Feature Engineering the column which will give the number of days a customer has been a member

In [37]:
days_mem = []

for days in dates:
    days_member = max_date - days
    days_mem.append(days_member)
    
df_orig['Num_days_member'] = days_mem

In [38]:
df_orig['Num_days_member'] = df_orig['Num_days_member'].astype(str).str.extract(r'(\d+)').fillna('0').astype(int)
# df_orig['Num_days_member'] = df_orig['Num_days_member'].astype(str).str.extract(r'(\d+.\d+)').fillna('0').astype(float)

In [39]:
df_orig.head()
# df_orig.info()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Num_days_member
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,0,0,0,0,0,0,3,11,1,663
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,0,0,0,0,0,0,3,11,0,113
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,0,0,0,0,0,0,3,11,0,312
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,0,0,0,0,0,0,3,11,0,139
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,0,0,0,0,0,0,3,11,0,161


In [40]:
df_orig.Marital_Status.value_counts()

Married     857
Together    573
Single      471
Divorced    232
Widow        76
Alone         3
Absurd        2
YOLO          2
Name: Marital_Status, dtype: int64

In [41]:
df_orig.Education.value_counts()

Graduation    1116
PhD            481
Master         365
2n Cycle       200
Basic           54
Name: Education, dtype: int64

#### Other column manipulations 
- Find age and keep that as a column seperate
- Create a feature IsParent (Boolean) which will convey the parenthood status
- Total Children column
- Living with? Extract from Marital Status
- Converge lesser value counts in Education Column
- Create a column with Total_spent indicatting total money spent by customers over two years in various categories


### Feature Engineering

In [42]:
## Adding the column with age of customers
df_orig['Age'] = 2022-df_orig['Year_Birth']
df_orig

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Num_days_member,Age
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,0,0,0,0,0,3,11,1,663,65
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,0,0,0,0,0,3,11,0,113,68
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,0,0,0,0,0,3,11,0,312,57
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,0,0,0,0,0,3,11,0,139,38
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,0,0,0,0,0,3,11,0,161,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,2013-06-13,46,709,...,0,0,0,0,0,3,11,0,381,55
2236,4001,1946,PhD,Together,64014.0,2,1,2014-06-10,56,406,...,0,0,1,0,0,3,11,0,19,76
2237,7270,1981,Graduation,Divorced,56981.0,0,0,2014-01-25,91,908,...,1,0,0,0,0,3,11,0,155,41
2238,8235,1956,Master,Together,69245.0,0,1,2014-01-24,8,428,...,0,0,0,0,0,3,11,0,156,66


In [44]:
### Total spendings done on various items
df_orig['Spent'] = df_orig['MntWines'] + df_orig['MntFruits'] + df_orig['MntMeatProducts'] + df_orig['MntFishProducts'] + df_orig['MntSweetProducts']+df_orig['MntGoldProds']

In [48]:
## Deriving living situation as 'Alone' or 'Together'
### This will reduce the feature space by reducing number of classes and helps improve overall performance
df_orig['Living_with'] = df_orig['Marital_Status'].replace({
    'Married':'Partner','Together':'Partner','Absurd':'Alone',
    'Divorced':'Alone','Widow':'Alone','YOLO':'Alone','Single':'Alone'
})
df_orig[['Living_with','Marital_Status']]

,Living_with,Marital_Status
0,Alone,Single
1,Alone,Single
2,Partner,Together
3,Partner,Together
4,Partner,Married
...,...,...
2235,Partner,Married
2236,Partner,Together
2237,Alone,Divorced
2238,Partner,Together


In [49]:
df_orig['Living_with'].value_counts() ## we have a 66% and 33% split between two classes as opposed to earlier

Partner    1430
Alone       786
Name: Living_with, dtype: int64

In [50]:
df_orig['Marital_Status'].value_counts() ## Imbalance throughout 

Married     857
Together    573
Single      471
Divorced    232
Widow        76
Alone         3
Absurd        2
YOLO          2
Name: Marital_Status, dtype: int64

In [51]:
### Making a column with total number of children i.e., kids+teens
df_orig['Total Children'] = df_orig['Kidhome']+df_orig['Teenhome']

In [52]:
### Isparent Feature added
df_orig['IsParent'] = np.where(df_orig['Total Children'] >=1, 1, 0)
# df_orig['IsParent'] = np.where(df_orig['Total Children'] >=1, True value, ELSE value)